In [ ]:
from dotenv import load_dotenv
env_loaded = load_dotenv()
env_loaded

In [3]:
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain_postgres import PGVector
import os

azure_deployment = "gpt-4o"


model = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_deployment=azure_deployment,
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=0,
    max_tokens=8192,
)


In [9]:
import requests
from urllib.parse import quote
from datetime import datetime



API_URL = "http://192.168.80.162/polarion/rest/v1/projects/ajmal_Training/workitems"

headers = {
    "Authorization": f"Bearer {os.getenv('POLARION_ACCESS_TOKEN')}",
    "Content-Type": "application/json",
}


def get_current_datetime():
    """
    Returns the current date and time.

    :return: A string representing the current date and time in the format YYYY-MM-DD HH:MM:SS.
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


def extract_workitems_from_polarion(lucene_query: str) -> str:
    """
    Fetch workitems based on the provided lucene query.

    Args:
        lucene_query (str): Lucene query that retrieves the relevant work items.

    Returns:
        str: Markdown-formatted table containing work item details
    """
    try:
        params = {"query": lucene_query, "fields[workitems]": "@all"}
        # Make API request
        response = requests.get(API_URL, headers=headers, params=params)
        response.raise_for_status()  # Raise exception for bad status codes

        # Parse response JSON
        json_data = response.json()

        # Extract work items details
        if "data" in json_data and isinstance(json_data["data"], list):
            # Prepare markdown table
            table_header = "| ID | Title | Description |\n|---|---|---|\n"
            table_rows = []

            for item in json_data["data"]:
                # Extract ID
                work_item_id = item.get("id", "")

                # Extract attributes (title and description)
                attributes = item.get("attributes", {})
                title = attributes.get("title", "")
                description = attributes.get("description", "")

                # Escape special markdown characters
                if title:
                    title = title.replace("|", "\\|")
                if description:
                    description = description["value"].replace("|", "\\|")

                # Create table row
                if work_item_id:
                    table_row = f"| {work_item_id} | {title} | {description} |\n"
                    table_rows.append(table_row)

            # Combine header and rows
            if table_rows:
                formatted_output = (
                    f"PORTAL_URL: {json_data['links']['portal']}\n\n"
                    + table_header
                    + "".join(table_rows)
                )
                return formatted_output
            else:
                return "No work items found in the response"
        else:
            return "Invalid response format: 'data' array not found"

    except requests.exceptions.RequestException as e:
        return f"API request failed: {str(e)}"
    except (KeyError, TypeError) as e:
        return f"Error parsing response: {str(e)}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"

Test the tool

In [ ]:
result = extract_workitems_from_polarion(
    "type:systemrequirement AND created:[20250115 TO 20250130]"
)

print(result)

extract_workitems_from_polarion(
    "type:systemrequirement AND created:[20250115 TO 20250130]"
)

In [4]:
SYSTEM_PROMPT = """You are an expert in interpreting user queries and generates helpful response based on the configured rules.


## Polarion supported object types:

| ID | NAME |
|----|------|
| systemrequirement | System Requirement |
| systemtestcase | System Test Case |
| softwarerequirement | Software Requirement |
| softwaretestcase | Software Test Case |
| risk | Risk |
| release | Release |
| workpackage | Work Package |
| task | Task |
| changerequest | Change Request |
| issue | Issue |
| testcase | Test Case |
| unittestcase | Unit Test Case |
| question | Question |
| systemTestCase1 | System Test Case 1 |
| systemRequirment1 | System Requirement 1 |
| new_wl_type | New Custom Type |

## Rules:

1. Understand the user query and if it is related to Polarion application data extraction, generate appropriate lucene query and use provided tools to fetch data.
2. From the provided Polarion object types, use values under ID column to formulate Lucene query based on user question. The NAME column is for your reference to interpret the user question.
3. If the user intent is to extract data that is not part of the provided object types, inform the user politely about the same.
4. If the user query is related to Polarion Application, then you are allowed to respond in that context. ALL OTHER queries should be politely rejected.
5. PORTAL_URL is for overall data and do not link it to a particular extracted data.

## Here are some of the examples for Lucene query for your understanding:

Example 1:
If user gives *Human Readable Query*: "Show all Task Workitems Assign to me"
and *UserId*: "458963"  then you should generate like *Lucene Query*: "type:task AND assignee.id:458963"

Example 2:
*Human Readable Query*: "Render Software Workitems that have specific comment"
*Lucene Query*: "type:softwarerequirement AND HAS_VALUE:comments.text"

Example 3:
*Human Readable Query*: "Display the Workitems which have Initial Estimate 1Day And Time Spent on a workitem should be 9h"
*Lucene Query*: "initialEstimate:1d AND timeSpent:9h"

Example 4:
*Human Readable Query*: "How to get workitems of type task & voice with open status"
*Lucene Query*: "type:(task voice) AND status:open"

Example 5:
*Human Readable Query*: "Show all Workitems created from 10-july-2024 to 30-Aug-2024"
*Lucene Query*: "created:[20240710 TO 20240830]"

Example 6:
*Human Readable Query*: "Filter Workitems based on Must Have Severity"
*Lucene Query*: "severity:must_have"

Example 7:
*Human Readable Query*: "Workitem which is assigned to me and status open and priority medium"
*Lucene Query*: "assignee.id:UserId AND status:open AND priority:medium"

Example 8:
*Human Readable Query*: "Workitem which are not assigned to me and status open and priority medium"
*Lucene Query*: "-assignee.id:UserId AND status:open AND priority:medium"  do not mention negative for the not mention please

## Response format:

Format your response in a neat well-structured markdown format.

## Example Response:

Here are the extracted work items:

workitem-5897
workitem-5693
worktitem-5869

You can view the extracted work items on the Polarion portal using this [link](PORTAL_URL)

"""

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import MessagesState
from langchain_core.messages import SystemMessage, HumanMessage
from typing import Literal
from pydantic import BaseModel
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode


sys_msg = SystemMessage(content=SYSTEM_PROMPT)
tools = [extract_workitems_from_polarion, get_current_datetime]
model_with_tools = model.bind_tools(tools, parallel_tool_calls=False)

# Node
def assistant(state: MessagesState):
    return {"messages": [model_with_tools.invoke([sys_msg] + state["messages"])]}


# Build graph
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")

graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [6]:
messages = [HumanMessage("Extract all system requirments created on 28-JAn-2025")]
# messages = [HumanMessage("How do I create LiveDocs ?")]

## Non Streaming

In [ ]:
result = graph.invoke({"messages": messages})

for message in result["messages"]:
    message.pretty_print()